In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
import requests
import pprint
import os

load_dotenv()
api_url = os.getenv("API_URL")
print(api_url)
# Define the login URL
login_url = api_url + "/api/auth/v1/login/"

# Define the payload with your email and password
payload = {
    "email": os.getenv("API_EMAIL"),
    "password": os.getenv("API_PASSWORD")
}

# Make the POST request to the login endpoint
response = requests.post(login_url, json=payload)
token = ""
headers = None
# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    response_data = response.json()
    print("Login successful!")
    print("Response data:")
    pprint.pp(response_data)
    token = response_data.get("access")
    headers = {"Authorization": f"Bearer {token}"}
else:
    print("Login failed!")
    print("Status code:", response.status_code)
    print("Response:", response.text)
    

http://picolo:8000


# Test for querying all catchments

In [ ]:
protected_response = requests.get(api_url + "/api/catchments/v1/getCatchments/", headers=headers)
json_data = None
if protected_response.status_code == 200:
    print("Accessed protected endpoint successfully!")
    json_data = protected_response.json()
    #print("Response data:", json_data)   
else:
    print("Failed to access protected endpoint.")
    print("Status code:", protected_response.status_code)
    #print("Response:", protected_response.text)

In [ ]:
import pandas as pd
catchments = json_data['results']

# Convert the results to a DataFrame
df_catchments = pd.DataFrame(catchments)
print(df_catchments)

# Example fetching measurement data from one catchment

This example demonstrates a simple query to extract date filtered data for one catchment. 

In [ ]:
protected_url = api_url + "/api/measurements/v2/getMeasurementsByTypeID/?Catchments=Catchment%201&from_date=2018-05-01&to_date=2018-06-31"

protected_response = requests.get(protected_url, headers=headers)
json_data = None
if protected_response.status_code == 200:
    print("Accessed protected endpoint successfully!")
    json_data = protected_response.json()
    #print("Response data:", json_data)   
else:
    print("Failed to access protected endpoint.")
    print("Status code:", protected_response.status_code)
    #print("Response:", protected_response.text)

Review the root object keys from the returned JSON to see what we are dealing with

In [ ]:
for key,value in json_data.items():
    print(f"{key}: {str(value)[:150]}") # just show the first 150 characters

We can see we see the results returns:
- *count*: a count of the number of records
- *next*: a URL for the next page (if there is one)
- *previous*: a URL for the previous page (if there is one)
- *results*: the NWFP data.

To use this dataset we need to pull down all the pages. We cannot modify the page length.

# Processing page data to create a master dataset

1. set up an object to store all the accumlated response data

In [ ]:
all_data = []
all_data.extend(json_data['results']['catchments'])  # Extract catchments data from the first call

2. While thre is a next URL loop and query the API to get the next set of data then append to the all_data array

In [ ]:
next_url = json_data["next"]
while next_url:
    response = requests.get(next_url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        all_data.extend(data['results']['catchments'])  # Extract catchments data
        next_url = data.get('next')  # Update the URL to the next page
        print(next_url)
    else:
        print(f"Failed to fetch data from {url}")
        break
print(len(all_data))

Having retrieved all the data we need to reshape into a more usable dataframe format. 

In [ ]:
# Flatten the nested structure and extract required fields

data_list = []
#pprint.pp(all_data)

for catchment in all_data:
    for location in catchment['locations']:
        for type_info in location['types']:
            for value in type_info['values']:
                data_list.append({
                    #'catchment_id': catchment['id'],
                    #'location_id': location['id'],
                    'type': type_info['type'],
                    'date': value['date'],
                    'LocationX': value['LocationX'],
                    'LocationY': value['LocationY'],
                    'value': value['value']
                })

# Convert the flattened data to a DataFrame
df = pd.DataFrame(data_list)

# Pivot the DataFrame
pivot_df = df.pivot_table(index=['date', 'LocationX', 'LocationY'], columns='type', values='value').reset_index()

# Flatten the column hierarchy
pivot_df.columns.name = None

print(pivot_df)

Now plot the data

In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
# Plot the data
plt.figure(figsize=(14, 7))

# Plot precipitation
plt.plot(pivot_df['date'], pivot_df['precipitation'], label='Precipitation', color='blue')

# Plot soil moisture
plt.plot(pivot_df['date'], pivot_df['soil moisture @ 10cm depth'], label='Soil Moisture @ 10cm Depth', color='green')

# Add labels and title
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Precipitation and Soil Moisture Over Time')
plt.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()